In [ ]:
### Exercise One

In [3]:
def print_keyword(keywords_list):
    for k in keywords_list:
        print(k)

In [4]:
keywords = ['designing roof shells', 'structural art shell', 'structural engineering swiss', 'reinforced roof shells', 'isler structural art', 'shell construction basel', 'roof shells genau', 'roof shells trösch', 'shell roof construction', 'art structural engineering', 'construction process shells', 'model roofing hall', 'structural engineering', 'construction process isler', 'roof structure realised', 'construction shells', 'thermal insulation formwork', 'shell construction', 'significantly construction shells', 'shell roof', 'roof shells variant', 'arrangement roof shells', 'insulation formwork', 'factory building glass', 'perfection industrial shells', 'roof shells', 'formwork beams', 'model built structure', 'industrial shells', 'construction process trösch', 'isler structural artist', 'formwork beams functioned', 'classical shell shed', 'isler industrial shells', 'roof structure', 'constructed successively formwork', 'designing roof', 'factory building glas', 'industrial shells used', 'projects designing roof', 'built structure', 'shells formwork', 'bubble shells supports', 'produced special formwork', 'isler folded roof', 'thesis roofing', 'building glass', 'roofing hall', 'symposium concrete shell', 'structural artist exh', 'roof construction', 'shell supporting structure', '1956 surfaces shells', 'model roofing', 'construction basel birkhäuser', 'flat shells framed', 'concrete shell roof', 'formwork shell', 'isler engineer contribution', 'structural art', 'formwork perfecta acoustic', 'rectangular bubble shells', 'billington art structural', 'building consists', 'shell shed roof', 'surfaces shells', 'roofing hall building', 'industrial shells variant', 'oeuvre factory building', 'factory building bützberg', 'construction shells took', 'reconstruct construction process', 'roofing textile factory', 'factory building', 'structural artist heinz', 'regard construction process', 'art shell concrete', 'factory building sources', 'art structural', 'envisages arrangement shells', 'beams shell', 'reconstruct construction', 'shells renovated 1987', 'shells renovated', 'concrete shell', 'shell concrete billington', 'diploma thesis roofing', 'completion shells renovated', 'larger factory building', 'spiegelmanufaktur bützberg architect', 'building glas und', 'manufactory fr trösch', 'thesis roofing textile', 'arrangement roof', 'bubble shells 14', 'construction process', 'shells gives structure', 'shell surface', 'arrangement shells', 'building description later']

In [7]:
print_keyword(keywords)

designing roof shells
structural art shell
structural engineering swiss
reinforced roof shells
isler structural art
shell construction basel
roof shells genau
roof shells trösch
shell roof construction
art structural engineering
construction process shells
model roofing hall
structural engineering
construction process isler
roof structure realised
construction shells
thermal insulation formwork
shell construction
significantly construction shells
shell roof
roof shells variant
arrangement roof shells
insulation formwork
factory building glass
perfection industrial shells
roof shells
formwork beams
model built structure
industrial shells
construction process trösch
isler structural artist
formwork beams functioned
classical shell shed
isler industrial shells
roof structure
constructed successively formwork
designing roof
factory building glas
industrial shells used
projects designing roof
built structure
shells formwork
bubble shells supports
produced special formwork
isler folded roof


## Change the following print statement to print first 10 keywords:

In [9]:
print(keywords[:7])

['designing roof shells', 'structural art shell', 'structural engineering swiss', 'reinforced roof shells', 'isler structural art', 'shell construction basel', 'roof shells genau']


## Change the following print statement to print the 5th keyword. Remember that indexing starts with 0: 

In [10]:
print(keywords[7])

roof shells trösch
